In [1]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 23kB/s 
     |████████████████████████████████| 512kB 43.0MB/s 
     |████████████████████████████████| 3.8MB 41.3MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=79b3594bb6e07dbce0aeec8643acfebb02a8c6e48893aec4bbf3f4327e2c403e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import io
from pathlib import Path
# Authentication for Managing Data
from google.colab import drive
drive.mount('/content/drive')
tf.enable_eager_execution()

Mounted at /content/drive


Preprocessing Functions

In [69]:
def preprocess(dataset,seqLen = 50):
    sats = tf.sparse.to_dense(dataset['sats'])
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])

    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)
    # take indices only the indices where 
    # (1) the time at the beginning of a slice is less than the time at the end 
    # of slice (ensures continuity)
    # (2) the satellite index matches the provided one
    indices = tf.where((time[:-seqLen]<time[seqLen:]))
    zeros = tf.zeros_like(indices)
    # the indices need zeros in the second column
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset
    dsBegin = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels
    dataSlices = dsBegin.map(lambda x: tf.slice(data,x,[seqLen,24]))
    # select labels for data in the same way
    correctLabels = tf.boolean_mask(labels,(time[:-seqLen]<time[seqLen:]))
    correctLabels = tf.reshape((correctLabels > 0),(-1,1))
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    dsReturn = tf.data.Dataset.zip((dataSlices,dataLabels))
    return dsReturn

In [94]:
def preprocDetInd(dataset,seqLen = 50,sat=0):
    #Set the Satellite viewpoint
    satView = tf.constant(sat,dtype=tf.int64)
    
    sats = tf.sparse.to_dense(dataset['sats'])
    labels = tf.sparse.to_dense(dataset['label'])
    time = tf.sparse.to_dense(dataset['time'])

    x1  = tf.sparse.to_dense(dataset['x1'])
    y1  = tf.sparse.to_dense(dataset['y1'])
    z1  = tf.sparse.to_dense(dataset['z1'])
    vx1 = tf.sparse.to_dense(dataset['vx1'])
    vy1 = tf.sparse.to_dense(dataset['vy1'])
    vz1 = tf.sparse.to_dense(dataset['vz1'])

    x2  = tf.sparse.to_dense(dataset['x2'])
    y2  = tf.sparse.to_dense(dataset['y2'])
    z2  = tf.sparse.to_dense(dataset['z2'])
    vx2 = tf.sparse.to_dense(dataset['vx2'])
    vy2 = tf.sparse.to_dense(dataset['vy2'])
    vz2 = tf.sparse.to_dense(dataset['vz2'])

    x3  = tf.sparse.to_dense(dataset['x3'])
    y3  = tf.sparse.to_dense(dataset['y3'])
    z3  = tf.sparse.to_dense(dataset['z3'])
    vx3 = tf.sparse.to_dense(dataset['vx3'])
    vy3 = tf.sparse.to_dense(dataset['vy3'])
    vz3 = tf.sparse.to_dense(dataset['vz3'])

    x4  = tf.sparse.to_dense(dataset['x4'])
    y4  = tf.sparse.to_dense(dataset['y4'])
    z4  = tf.sparse.to_dense(dataset['z4'])
    vx4 = tf.sparse.to_dense(dataset['vx4'])
    vy4 = tf.sparse.to_dense(dataset['vy4'])
    vz4 = tf.sparse.to_dense(dataset['vz4'])


    data = tf.stack([x1,y1,z1,vx1,vy1,vz1,
                     x2,y2,z2,vx2,vy2,vz2,
                     x3,y3,z3,vx3,vy3,vz3,
                     x4,y4,z4,vx4,vy4,vz4])
    data = tf.transpose(data)

    indices = tf.where((time[:-seqLen]<time[seqLen:]) & tf.equal(sats[:-seqLen],satView))
    zeros = tf.zeros_like(indices)
    begin = tf.stack([indices,zeros],axis = 1)
    begin = tf.reshape(begin,tf.shape(begin)[:2])
    # Construct dataset
    ds = tf.data.Dataset.from_tensor_slices(begin)

    # Map dataset as sequence of length seq_len and labels

    dataSlices = ds.map(lambda x: tf.slice(data,x,[seqLen,24]))
    correctLabels = tf.boolean_mask(labels,(time[:-seqLen]<time[seqLen:]) & tf.equal(sats[:-seqLen],satView))
    correctLabels = tf.reshape((correctLabels > 0),(-1,1))
    dataLabels = tf.data.Dataset.from_tensor_slices(correctLabels)

    ds3 = tf.data.Dataset.zip((dataSlices,dataLabels))
    return ds3

In [23]:
def decode_TFRecord(exampleProto):
# Read TFRecord file
    # Define features
    featureDescription = {
        'x1': tf.VarLenFeature(dtype=tf.float32),
        'y1': tf.VarLenFeature(dtype=tf.float32),
        'z1': tf.VarLenFeature(dtype=tf.float32),
        'vx1': tf.VarLenFeature(dtype=tf.float32),
        'vy1': tf.VarLenFeature(dtype=tf.float32),
        'vz1':tf.VarLenFeature(dtype=tf.float32),
        'x2': tf.VarLenFeature(dtype=tf.float32),
        'y2': tf.VarLenFeature(dtype=tf.float32),
        'z2': tf.VarLenFeature(dtype=tf.float32),
        'vx2': tf.VarLenFeature(dtype=tf.float32),
        'vy2': tf.VarLenFeature(dtype=tf.float32),
        'vz2':tf.VarLenFeature(dtype=tf.float32),
        'x3': tf.VarLenFeature(dtype=tf.float32),
        'y3': tf.VarLenFeature(dtype=tf.float32),
        'z3': tf.VarLenFeature(dtype=tf.float32),
        'vx3': tf.VarLenFeature(dtype=tf.float32),
        'vy3': tf.VarLenFeature(dtype=tf.float32),
        'vz3': tf.VarLenFeature(dtype=tf.float32),
        'x4': tf.VarLenFeature(dtype=tf.float32),
        'y4': tf.VarLenFeature(dtype=tf.float32),
        'z4': tf.VarLenFeature(dtype=tf.float32),
        'vx4': tf.VarLenFeature(dtype=tf.float32),
        'vy4': tf.VarLenFeature(dtype=tf.float32),
        'vz4': tf.VarLenFeature(dtype=tf.float32),
        'label': tf.VarLenFeature(dtype=tf.int64),
        'time': tf.VarLenFeature(dtype=tf.int64),
        'sats': tf.VarLenFeature(dtype=tf.int64)}

    # Extract features from serialized data
    return  tf.io.parse_single_example(exampleProto, featureDescription)

In [6]:
rootPath = '/content/drive/My Drive/Data/'

register = np.zeros(1)
while not np.any(register):
    try:
        with open(rootPath + 'Isolation/Testing/FileRegisterClosed.csv','r') as f:
            register = np.genfromtxt(f,delimiter = ",")
    except:
        pass
np.shape(register)


(3565, 2)

In [87]:
# Parameter Definitions

path = '/content/drive/My Drive/Data/Detection/Testing/'
# Define model parameters
nUnits = [50,50]
nTimesteps = 50

# Create Dataset
cn = ['x','y','z','vx','vy','vz']
file = '/content/drive/My Drive/Data/Detection/Testing/TestFault_1_12_0.15013000.tfrecord'
listdir = [file,file]

In [95]:
fileListDataset = tf.data.TFRecordDataset(listdir)
decodedDataset = fileListDataset.map(decode_TFRecord)
processedDataset = decodedDataset.flat_map(preprocess)


In [96]:
datalabels = ['x1','y1','z1','vx1' ,'vy1','vz1','x2','y2','z2','vx2','vy2','vz2','x3',
        'y3','z3','vx3','vy3','vz3','x4','y4','z4','vx4','vy4','vz4','label','time','sats']

# Take data and labels of entire run (just the decoded Dateset)
npdataFull = np.zeros([33612,27])
for data in decodedDataset.take(-1):
    for number, label in enumerate(datalabels):
        npdataFull[:,number] = tf.sparse.to_dense(data[label]).numpy()

# Take data from the processed Dataset and compare if it matches with the correct
# spot in the full set
i = 0
indexList = []
incorrectLabelList = []
for data,label in processedDataset.take(-1):
    if i < 33612 - 51:
        npdata = data.numpy()
        diff = npdataFull[i:i+50,0:24] - npdata
        while np.abs(np.sum(diff)) > 1e-9 :
            indexList.append(i)
            i += 1 
            diff = npdataFull[i:i+50,0:24] - npdata
        if np.abs(np.sum(diff)) < 1e-9 :
            if not label.numpy() == npdataFull[i,-3]:
                print(np.sum(diff))
                print(label.numpy(),npdataFull[i,-3])
                incorrectLabelList.append(i)
        i += 1 
print(indexList[:10])
if len(indexList) == 250 and len(incorrectLabelList) == 0:
    print("Slicing Preprocessing Successful")


[5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561]
Slicing Preprocessing Successful
[5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561]
Slicing Preprocessing Successful


In [116]:
# Check Satellite Selection
satView = 1
satProcessedDataset = decodedDataset.flat_map(lambda x: preprocDetInd(x,50,satView))
i0 = np.where(npdataFull[:,-1] == satView)[0][0]
i = i0
indexList = []

for data,label in satProcessedDataset.take(-1):
    if i < i0 + 33612//6:
        npdata = data.numpy()
        diff = npdataFull[i:i+50,0:24] - npdata
        while np.abs(np.sum(diff)) > 1e-9 and i < i0 + 33612//6:
            indexList.append(i)
            i = i+1
            diff = npdataFull[i:i+50,0:24] - npdata

    i += 1 
print(indexList[:10])
if len(indexList) == 50:
    print("Satellite Selection Successful")



ValueError: ignored

In [110]:
 len(indexList)

50